In [1]:
import sys
sys.path.append("/home/arc/Development/DeepHypergraph/") 

In [2]:
import time
from copy import deepcopy

import torch
import torch.optim as optim
import torch.nn.functional as F

from dhg import Hypergraph
from dhg.data import DBLP8k
from dhg.models import HGNNPLinkPred
from dhg.random import set_seed
from dhg.metrics import LinkPredictionEvaluator as Evaluator

In [3]:
def train(net, X, hypergraph, negative_hypergraph, optimizer, epoch):
    net.train()

    st = time.time()
    optimizer.zero_grad()
    pos_score = net(X, hypergraph)
    neg_score = net(X, negative_hypergraph)

    scores = torch.cat([pos_score, neg_score]).squeeze()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).to(device)

    loss = F.binary_cross_entropy_with_logits(scores, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Time: {time.time()-st:.5f}s, Loss: {loss.item():.5f}")
    return loss.item()

In [4]:
@torch.no_grad()
def infer(net, X, hypergraph, negative_hypergraph, test=False):
    net.eval()
    pos_score = net(X, hypergraph)
    neg_score = net(X, negative_hypergraph)

    scores = torch.cat([pos_score, neg_score]).squeeze()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    ).to(device)

    if not test:
        res = evaluator.validate(labels, scores)
    else:
        res = evaluator.test(labels, scores)
    return res

In [5]:
import csv
from pathlib import Path

def load_data(file_path: Path):
    hyperedge_list = []
    neg_hyperedge_list = []
    with open(file_path / "hyperedges.csv", "r") as file:
        reader = csv.reader(file)
        for row in reader:
            # 读取每个超边的顶点列表，并将它们添加到 hyperedge_list 中
            hyperedge_list.append(row)
    
    hyperedge_list = [[int(v) for v in edge] for edge in hyperedge_list]

    with open(file_path / "minimal_unschedulable_combinations.csv", "r") as file:
        reader = csv.reader(file)
        for row in reader:
            neg_hyperedge_list.append(row) 

    neg_hyperedge_list = [[int(v) for v in edge] for edge in neg_hyperedge_list]

    with open(file_path / "task_quadruples.csv", 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = [list(map(float, row)) for row in reader]

    # 将数据转换为 Tensor
    features = torch.tensor(data)

    data = {"hyperedge_list": hyperedge_list, "num_edges" : len(hyperedge_list)}
    neg_data = {"hyperedge_list": neg_hyperedge_list, "num_edges" : len(neg_hyperedge_list)}

    return {"pos":data, "neg": neg_data, "vertices_feature" : features, "num_vertices" : features.shape[0]}

In [6]:
set_seed(2021)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
evaluator = Evaluator(["accuracy", "auc"])
data = load_data(Path("../EDF/data/data_s2233_p10_t21/"))

X = data["vertices_feature"]
HG = Hypergraph(data["num_vertices"], data["pos"]["hyperedge_list"])
neg_HG = Hypergraph(data["num_vertices"], data["neg"]["hyperedge_list"])

In [7]:
import numpy as np

def calculate_sparsity(matrix):
    nonzero_elements = np.count_nonzero(matrix)
    total_elements = matrix.size

    nonzero_ratio = nonzero_elements / total_elements
    zero_ratio = 1 - nonzero_ratio

    print(f"非零元素比例：{nonzero_ratio:.2%}")
    print(f"零元素比例：{zero_ratio:.2%}")

# calculate_sparsity(HG.H.to_dense().cpu().numpy())

In [8]:
net = HGNNPLinkPred(X.shape[1], 64, 32, use_bn=True)
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

In [9]:
X = X.to(device)
HG = HG.to(device)
neg_HG = neg_HG.to(device)
net = net.to(device)

In [10]:
print(f"X: {X.device}")
print(f"HG: {HG.device}")
print(f"neg_HG: {neg_HG.device}")
print(f"net: {next(net.parameters()).device}")

X: cuda:0
HG: cuda
neg_HG: cuda
net: cuda:0


In [11]:
best_state = None
best_epoch, best_val = 0, 0
for epoch in range(200):
    # train
    train(net, X, HG, neg_HG, optimizer, epoch)
    # validation
    if epoch % 1 == 0:
        with torch.no_grad():
            val_res = infer(net, X, HG, neg_HG)
        if val_res > best_val:
            print(f"update best: {val_res:.5f}")
            best_epoch = epoch
            best_val = val_res
            best_state = deepcopy(net.state_dict())
print("\ntrain finished!")
print(f"best val: {best_val:.5f}")


Epoch: 0, Time: 13.94934s, Loss: 0.71034
Epoch: 1, Time: 0.14803s, Loss: 0.70291
Epoch: 2, Time: 0.18896s, Loss: 0.69711
Epoch: 3, Time: 0.08502s, Loss: 0.68992
Epoch: 4, Time: 0.02663s, Loss: 0.68374
Epoch: 5, Time: 0.02426s, Loss: 0.68464
Epoch: 6, Time: 0.02366s, Loss: 0.67570
Epoch: 7, Time: 0.02569s, Loss: 0.68786
Epoch: 8, Time: 0.02613s, Loss: 0.66394
Epoch: 9, Time: 0.01943s, Loss: 0.67791
Epoch: 10, Time: 0.02061s, Loss: 0.67099
Epoch: 11, Time: 0.02417s, Loss: 0.66806
Epoch: 12, Time: 0.02282s, Loss: 0.65294
Epoch: 13, Time: 0.02283s, Loss: 0.65798
Epoch: 14, Time: 0.02133s, Loss: 0.65419
Epoch: 15, Time: 0.02254s, Loss: 0.64639
Epoch: 16, Time: 0.01966s, Loss: 0.64365
Epoch: 17, Time: 0.02238s, Loss: 0.63812
Epoch: 18, Time: 0.02263s, Loss: 0.67245
Epoch: 19, Time: 0.02671s, Loss: 0.64139
Epoch: 20, Time: 0.02778s, Loss: 0.61383
Epoch: 21, Time: 0.02134s, Loss: 0.61055
Epoch: 22, Time: 0.02038s, Loss: 0.63428
Epoch: 23, Time: 0.02267s, Loss: 0.60059
Epoch: 24, Time: 0.01989s

In [12]:
# test
print("test...")
net.load_state_dict(best_state)
res = infer(net, X, HG, neg_HG, test=True)
print(f"final result: epoch: {best_epoch}")
print(res)

test...
final result: epoch: 32
{'accuracy': 0.5383862257003784, 'auc': 1.0}
